# MatrixPerspective.jl

In [1]:
versioninfo()

Julia Version 1.5.1
Commit 697e782ab8 (2020-08-25 20:08 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin19.5.0)
  CPU: Intel(R) Core(TM) i5-8279U CPU @ 2.40GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-9.0.1 (ORCJIT, skylake)


## Timing

The following code compares the three methods, interior point method (by using MOSEK, for $p < 100$), bisection, and semismooth Newton, for finding the unique root of the semismooth function
$$
    f(\mu) = 1 - \mathbf{e}^T(\bar{\mathbf{X}} - \mu \mathbf{e}\mathbf{e}^T)_{+}\mathbf{e}
$$
where $\mathbf{e}=(0, \dotsc, 0, 1)^T$ and
$$
    \bar{\mathbf{X}} = \begin{bmatrix} -\mathbf{X} & \frac{1}{\sqrt{2}}\mathbf{y} \\
                        \frac{1}{\sqrt{2}}\mathbf{y}^T & 1 \end{bmatrix}
$$
for given input $(\mathbf{X}, \mathbf{y})$. From this root the prox operator
$$
    \mathrm{prox}_{\phi}(\mathbf{X}, \mathbf{y}),
    \quad
    \phi(\boldsymbol{\Omega}, \boldsymbol{\eta}) = \begin{cases}
    \frac{1}{2}\boldsymbol{\eta}^T\boldsymbol{\Omega}^{\dagger}\boldsymbol{\eta}, &
    \boldsymbol{\Omega} \succeq \mathbf{0},~\boldsymbol{\eta} \in \mathcal{R}(\boldsymbol{\Omega}) \\
    \infty, & \text{otherwise}
    \end{cases}
$$
can be computed in a closed form.

The first table reports the mean of the performance measures, and the second table contains the standard deviation. The $p=5$ case (especially for MOSEK) should be ignored since there is an overhead of JIT compilation of the code.

In [3]:
;cat timing.jl

using MatrixPerspective

using LinearAlgebra
import LinearAlgebra.BLAS.BlasInt

using Random
using DataFrames, Statistics

Random.seed!(1234);

reps = 10 #reps = 100
tol = 1e-8   # default

# KKT measures |g'(mu)|
df = DataFrame(p=Int[], Method=String[], Iters=Float64[], Secs=Float64[], KKT=Float64[], Obj=Float64[])
# MOSEK stalls if p > 30
dims = [5, 10, 30, 50, 100, 500, 1000, 2000]  
#dims = [10, 30] #dims = [100] #dims = [1000]
nummethods = 2
Means = zeros(nummethods, length(dims), size(df)[2]);
mosek_maxdim = 100
for i = 1:length(dims)
	p = dims[i]
	n = p + 2

	e = zeros(p + 1)  # last elementary unit vector
	e[end] = 1

	# workspaces
	Q = Matrix{Float64}(undef, n, n)
	evec = Vector{Float64}(undef, n)

	d = Vector{Float64}(undef, n)

	indxq = Vector{BlasInt}(undef, n)

	z = Vector{Float64}(undef, n)
	dlambda = Vector{Float64}(undef, n)
	w = Vector{Float64}(undef, n)
	Q2 = Vector{Float64}(undef, n * n)

	indx   = Vector{BlasInt}(undef, n + 1)
	indxc  = Vector{BlasInt}(undef, n)
	in

In [4]:
include("timing.jl")

p = 5
p = 10
p = 30
p = 50


┌ Warning: Problem status SLOW_PROGRESS; solution may be inaccurate.
└ @ Convex /Users/jhwon/.julia/packages/Convex/aYxJA/src/solution.jl:252


p = 100


┌ Warning: Problem status SLOW_PROGRESS; solution may be inaccurate.
└ @ Convex /Users/jhwon/.julia/packages/Convex/aYxJA/src/solution.jl:252
┌ Warning: Problem status SLOW_PROGRESS; solution may be inaccurate.
└ @ Convex /Users/jhwon/.julia/packages/Convex/aYxJA/src/solution.jl:252
┌ Warning: Problem status SLOW_PROGRESS; solution may be inaccurate.
└ @ Convex /Users/jhwon/.julia/packages/Convex/aYxJA/src/solution.jl:252
┌ Warning: Problem status SLOW_PROGRESS; solution may be inaccurate.
└ @ Convex /Users/jhwon/.julia/packages/Convex/aYxJA/src/solution.jl:252
┌ Warning: Problem status SLOW_PROGRESS; solution may be inaccurate.
└ @ Convex /Users/jhwon/.julia/packages/Convex/aYxJA/src/solution.jl:252
┌ Warning: Problem status SLOW_PROGRESS; solution may be inaccurate.
└ @ Convex /Users/jhwon/.julia/packages/Convex/aYxJA/src/solution.jl:252
┌ Warning: Problem status SLOW_PROGRESS; solution may be inaccurate.
└ @ Convex /Users/jhwon/.julia/packages/Convex/aYxJA/src/solution.jl:252


p = 500
p = 1000
p = 2000
24×6 DataFrame
│ Row │ p     │ Method    │ Iters_mean │ Secs_mean   │ KKT_mean    │ Obj_mean  │
│     │ Int64 │ String    │ Float64    │ Float64     │ Float64     │ Float64   │
├─────┼───────┼───────────┼────────────┼─────────────┼─────────────┼───────────┤
│ 1   │ 5     │ MOSEK     │ NaN        │ 2.49323     │ 1.21816e-5  │ 6.53939   │
│ 2   │ 5     │ Bisection │ 28.4       │ 0.157635    │ 5.97528e-9  │ 6.53939   │
│ 3   │ 5     │ Newton    │ 4.2        │ 0.0943318   │ 5.58998e-12 │ 6.53939   │
│ 4   │ 10    │ MOSEK     │ NaN        │ 0.00983312  │ 1.828e-5    │ 27.5118   │
│ 5   │ 10    │ Bisection │ 29.4       │ 0.000295918 │ 4.56144e-9  │ 27.5118   │
│ 6   │ 10    │ Newton    │ 4.9        │ 0.000165922 │ 2.34184e-10 │ 27.5118   │
│ 7   │ 30    │ MOSEK     │ NaN        │ 0.102232    │ 4.92603e-6  │ 225.301   │
│ 8   │ 30    │ Bisection │ 31.8       │ 0.001258    │ 5.44707e-9  │ 225.301   │
│ 9   │ 30    │ Newton    │ 5.7        │ 0.000554437 │ 1.18062e-9  │

## PDHG
The following code illustrates how to use `prox_matrixperspective!()` for the PDHG algorithm for Gaussian joint likelihood estimation.

In [5]:
;cat gaussianmle.jl

using MatrixPerspective

using LinearAlgebra
import LinearAlgebra.BLAS.BlasInt

using IterativeSolvers

# Joint MLE of multivariate Gaussian natural parameters
# PDHG code based on http://proximity-operator.net/tutorial.html

function gaussianmle(X::Matrix{T}, cvec::Array{Vector{T},1}; 
					 ep::T = 10 / size(X)[2]^2, 
					 init::Matrix{T} = Array{Float64}(undef, 0, 0), 
					 maxiter::Integer = 1000, 
					 opttol::T = 1e-4, 
					 log::Bool = false
					) where T <: Float64
	n, p = size(X)  # sample size, dimension
	m = length(cvec)

	# workspaces
	_n = p + 2
	_Q = Matrix{Float64}(undef, _n, _n)
	_evec = Vector{Float64}(undef, _n)

	_d = Vector{Float64}(undef, _n)

	_indxq = Vector{BlasInt}(undef, _n)

	_z = Vector{Float64}(undef, _n)
	_dlambda = Vector{Float64}(undef, _n)
	_w = Vector{Float64}(undef, _n)
	_Q2 = Vector{Float64}(undef, _n * _n)

	_indx   = Vector{BlasInt}(undef, _n + 1)
	_indxc  = Vector{BlasInt}(undef, _n)
	_indxp  = Vector{BlasInt}(undef, _n)
	_coltyp = Vector{Bla

Function `gaussianmle()` is called as follows.

In [6]:
;cat testgaussianmle.jl

include("gaussianmle.jl")
using Random, LinearAlgebra, SparseArrays

Random.seed!(123)
n, p = 60, 100
# data matrix
pcov = 0.3 * ones(p, p) + 0.7 * I  # covariance matrix
pchol = cholesky(pcov)
X = randn(n, p) * pchol.U   # correlated predictors
S = X' * X / n # second monent
mu = reshape(sum(X, dims=1) / n , p )   # sample mean

# Variance constraints
# cvec should be scaled so that
#  cvec[i]' * (Omega \ cvec[i]) <= 1
cvec = Array{Vector{Float64}, 1}()
m = 5
for i=1:m
	e = zeros(p)
	e[i] = 1.0
	push!(cvec, e)
end

maxit = 5000
@time eta, Omega, Y, history = gaussianmle(X, cvec, maxiter=maxit, opttol=1e-5)



In [7]:
include("testgaussianmle.jl")

objval = 824.1079588444118
it = 100
objval = -68.55345451618902
it = 200
objval = -90.7432608163429
it = 300
objval = -106.47412453177637
it = 400
objval = -119.48174383805825
it = 500
objval = -130.905231884441
it = 600
objval = -141.24124058414466
it = 700
objval = -150.75060287270733
it = 800
objval = -159.58819651238332
it = 900
objval = -167.8548291740341
it = 1000
objval = -175.62112227294827
it = 1100
objval = -182.93963829174476
it = 1200
objval = -189.85149972890522
it = 1300
objval = -196.39020970969847
it = 1400
objval = -202.58396017158287
it = 1500
objval = -208.45708168793593
it = 1600
objval = -214.03098642732363
it = 1700
objval = -219.32480171804784
it = 1800
objval = -224.3558092685453
it = 1900
objval = -229.13975914037536
it = 2000
objval = -233.6911010552277
it = 2100
objval = -238.02315987101295
it = 2200
objval = -242.1482724796969
it = 2300
objval = -246.07789742449864
it = 2400
objval = -249.82270476506264
it = 2500
objval = -253.39265129471005
it = 2600
objval

([-116.11434098621383, 48.95213796995712, 90.65112808590104, 74.88200654160379, -109.65229977959252, -98.84490753993417, -30.09665095952194, 136.9517017393632, -104.99144451079381, -18.399580485564563  …  -49.91108946571656, 117.59080136676128, -68.00851618779222, -94.5335199879541, 74.52083217154109, 90.99154106488278, 108.42836157637981, 133.7807753285948, -48.671229175687735, -102.01022091106664], [21.33766714582228 -2.6238047353834855 … 0.6781649941396397 4.81120439765941; -2.6238047353834855 12.248940051450315 … -2.527585193176585 -3.467548444667807; … ; 0.6781649941396397 -2.527585193176585 … 17.906439248985176 3.2836072748514855; 4.81120439765941 -3.467548444667807 … 3.2836072748514855 21.367264541224216], [[-0.14575876261675827 -0.03563277907757659 … -0.03404245270484068 -0.06075941502196815; -0.03563277907757659 -0.008710933888275197 … -0.008322156244423087 -0.014853493357741087; … ; -0.03404245270484068 -0.008322156244423087 … -0.007950730133517705 -0.014190567175007886; -0.0